# Recuperação de dados no Twitter

Este notebook tem como objetivo demostrar a recuperação de dados no tweeter, via API e persistir as informações em documentos para posterior uso.

## Imports

Vamos utilizar nesse exemplo as bibliotecas:

* os: Para acesso aos recursos do sistema operacional
* tweepy: Para acesso ao twitter (login, coleta)
* pandas: Para manipulação dos dados no formato tabular

In [1]:
import os
import tweepy
import pandas as pd

## Carregamento das chaves de acesso

As chaves de acesso podem ser geradas através do portal do desenvolvedor do Twitter.

* Site do Twitter: https://twitter.com/ (Links para um site externo.)

* Área de desenvolvedor https://developer.twitter.com/en/apply/user (Links para um site externo.)

* Área de aplicativos https://developer.twitter.com/en/apps (Links para um site externo.)

In [2]:
consumer_key =  os.environ['tweety_consumer_key']
consumer_secret = os.environ['tweety_consumer_secret']
access_token = os.environ['tweety_access_token']
acess_token_secret = os.environ['tweety_acess_token_secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, acess_token_secret)
api = tweepy.API(auth)

## Dataframe

Vamos utilizar um dataframe para persistir nossos registros obtidos a partir do stream de twetts. O Dataframe é similar a uma planilha.

In [3]:
df = pd.DataFrame(columns=['name', 'description', 'created_at', 'text', 'location', 'retweeted', 'owner_name', 'owner_location', 'owner_created_at', 'retweeted_count'])

df.head()

,name,description,created_at,text,location,retweeted,owner_name,owner_location,owner_created_at,retweeted_count


## Streamlistener

Vamos criar um stream listener que ira receber os twittes de acordo com a pesquisa que fizermos. Para isso vamos extender uma classe do pacote tweepy e fazer nossa implementação.

In [4]:
class StreamListner(tweepy.StreamListener):
    def __init__(self, df):
        super().__init__()
        self.__df__ = df

    def on_status(self, status):
        json = dict(status._json)
        user = json['user']
        
        retweety = json.get('retweeted_status')
        retweety_user = retweety['user'] if retweety != None and retweety.get('user') != None else None

        obj = { 'name': user['name'], 'description': user['description'], 'created_at': json['created_at'], 'text': json['text'], 'location': user.get('location'), 'retweeted': retweety != None }

        if retweety != None:
            obj['owner_name'] = retweety_user['name']
            obj['owner_location'] = retweety_user.get('location')
            obj['owner_created_at'] = retweety['created_at']
            obj['retweeted_count'] = retweety.get('retweet_count', 0)
        
        self.__df__ = self.__df__.append(obj, ignore_index=True)
        

    def on_error(self, status_code):
        if status_code == 420:
            return False

    @property
    def df(self):
        return self.__df__

### Definir o(s) termo(s) de pesquisa.

Podemos definir os n termos que serão utilizados para pesquisa

In [5]:
track = ['covid19', 'covid-19', 'sars-cov-2', 'coronavirus', 'corona-virus', 'coronavac', 'vacina']

Criamos o objeto responsável pela busca dos twittes

In [6]:
stream_listner = StreamListner(df)
stream = tweepy.Stream(auth=api.auth, listener=stream_listner)
stream.filter(track=track, languages=['pt'], is_async=True)

## Visualização dos registros - Obtidos pelo stream

Vamos visualizar os registros obtidos pelo stream atraves do dataframe

In [12]:
df = stream_listner.df
df

,name,description,created_at,text,location,retweeted,owner_name,owner_location,owner_created_at,retweeted_count
0,lore de áries,"Sofrendo por séries e casais desde 1997 , fã d...",Mon Feb 01 14:05:37 +0000 2021,"RT @drysanaoliveira: Não bastando a pandemia, ...",Grey Sloan Memorial Hospital,True,Capitu,"Manaus, Brasil",Mon Feb 01 01:49:23 +0000 2021,5
1,_ouriki_ツ,n me julgue :v,Mon Feb 01 14:05:37 +0000 2021,@videovergonhakk O meu filho n toma vacina é t...,None,False,NaN,NaN,NaN,NaN
2,Pablo Verdolaga 🇧🇷,PERFIL FOCADO EM ASSUNTOS POLÍTICOS E OPINIÕES...,Mon Feb 01 14:05:38 +0000 2021,RT @brunaobarreto3: Não pode:\n\n- Nomear como...,Brasil 🇧🇷,True,Brunão,None,Mon Feb 01 12:23:00 +0000 2021,115
3,kátia mano,"Dentista, crocheteira, mãe, atéia, petista e s...",Mon Feb 01 14:05:38 +0000 2021,@roberta_bastoss Eu acredito. Mas a diferença...,"Copacabana, Rio de Janeiro",False,NaN,NaN,NaN,NaN
4,carolina,777,Mon Feb 01 14:05:38 +0000 2021,eu e os meus amigos em 2075 quando finalmente ...,None,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1137,dtom,Tudo postado inclui opinião pessoal,Mon Feb 01 14:19:10 +0000 2021,RT @DOprimido: E pensar que tinha governante i...,None,True,Demagogia do Oprimido 🔥,None,Mon Feb 01 13:32:44 +0000 2021,7
1138,Danielle Almeida,hi,Mon Feb 01 14:19:11 +0000 2021,RT @FMouraBrasil: Ao lado do negacionista Bols...,None,True,Felipe Moura Brasil,Rio de Janeiro,Mon Feb 01 14:13:32 +0000 2021,6
1139,n,🤸🏻‍♀️,Mon Feb 01 14:19:12 +0000 2021,"cara deveriam ter mais postos de vacinação, vi...","Manaus, Brasil",False,NaN,NaN,NaN,NaN
1140,Ailton Soff,"A vida é bela, a gente que enfeia ela!!!",Mon Feb 01 14:19:12 +0000 2021,RT @FMouraBrasil: Ao lado do negacionista Bols...,Brasil,True,Felipe Moura Brasil,Rio de Janeiro,Mon Feb 01 14:13:32 +0000 2021,7


In [13]:
df.to_csv('./datasets/corona.csv')

In [14]:
stream.disconnect()